In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pytesseract

In [4]:
# 2. Chargement des données (dataset)
def load_dataset(test):
    images = []
    labels = []
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            images.append(img)
            labels.append(folder)  # Le dossier correspond au caractère/nombre
    return np.array(images), np.array(labels)


In [5]:
# 3. Binarisation
def binarize_image(image):
    _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary

In [6]:
# 4. Dilatation
def dilate_image(binary_image):
    kernel = np.ones((3,3), np.uint8)
    dilated = cv2.dilate(binary_image, kernel, iterations=1)
    return dilated

In [7]:
# 5. Suppression des bordures
def remove_borders(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(image)
    cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)
    return mask

In [8]:
# 6. Détection de la plaque
def detect_plate(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = w / float(h)
        if 2 < aspect_ratio < 6 and w > 100 and h > 20:
            return x, y, w, h
    return None

In [9]:
# 7. Extraction de la plaque
def extract_plate(image, coords):
    if coords:
        x, y, w, h = coords
        plate = image[y:y+h, x:x+w]
        return plate
    return None

In [10]:
# 8. Segmentation des caractères
def segment_characters(plate):
    contours, _ = cv2.findContours(plate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    characters = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if 10 < w < 50 and 20 < h < 60:  # Filtrer selon la taille des caractères
            char = plate[y:y+h, x:x+w]
            char = cv2.resize(char, (28, 28))  # Redimensionner pour le modèle
            characters.append(char)
    return characters

In [11]:

# 9. Reconnaissance des caractères (préparation pour le CNN)
def preprocess_characters(characters):
    chars = np.array([char / 255.0 for char in characters])
    chars = chars.reshape(-1, 28, 28, 1)
    return chars


In [12]:
# 10. Matrice de convolution (intégrée dans le modèle CNN)
def build_resnet_model(num_classes):
    inputs = layers.Input(shape=(28, 28, 1))
    
    # Bloc ResNet simplifié
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    # Raccourci résiduel
    shortcut = layers.Conv2D(32, (1, 1), padding='same')(inputs)
    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [13]:
# 11. Intégration et post-traitement
def post_process(prediction, char_map):
    predicted_chars = [char_map[np.argmax(pred)] for pred in prediction]
    return ''.join(predicted_chars)


In [15]:
 #12. Création et entraînement du modèle
def train_model(images, labels, num_classes):
    # Prétraitement
    images = images.reshape(-1, 28, 28, 1) / 255.0
    label_map = {char: idx for idx, char in enumerate(np.unique(labels))}
    inverse_label_map = {idx: char for char, idx in label_map.items()}
    labels = np.array([label_map[label] for label in labels])
    

In [18]:
    # Séparation des données
dataset_path = "test"  # change selon ton dossier
images, labels = load_dataset(dataset_path)

# Puis maintenant ta séparation fonctionnera :
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (716,) + inhomogeneous part.

In [ ]:
    # Augmentation des données
    datagen = ImageDataGenerator(rotation_range=10, zoom_range=0.1, width_shift_range=0.1, height_shift_range=0.1)
    datagen.fit(X_train)

In [ ]:
   # Création et entraînement du modèle
    model = build_resnet_model(num_classes)
    model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=20, validation_data=(X_test, y_test))
    
    return model, inverse_label_map


In [ ]:
# 13. Prédiction
def predict_plate(image, model, inverse_label_map):
    binary = binarize_image(image)
    dilated = dilate_image(binary)
    cleaned = remove_borders(dilated)
    plate_coords = detect_plate(cleaned)
    plate = extract_plate(image, plate_coords)
    
    if plate is not None:
        characters = segment_characters(plate)
        if characters:
            chars_processed = preprocess_characters(characters)
            predictions = model.predict(chars_processed)
            plate_text = post_process(predictions, inverse_label_map)
            return plate_text
    return "No plate detected"

In [ ]:

# 14. Évaluation
def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {accuracy*100:.2f}%")
    return accuracy

In [ ]:
# Exemple d'utilisation
if __name__ == "__main__":
    # Remplacez par le chemin de votre dataset
    dataset_path = "path/to/your/dataset"
    images, labels = load_dataset(dataset_path)
    
    # Entraînement
    num_classes = len(np.unique(labels))
    model, inverse_label_map = train_model(images, labels, num_classes)
    
    # Prédiction sur une nouvelle image
    test_image = cv2.imread("test_plate.jpg", cv2.IMREAD_GRAYSCALE)
    result = predict_plate(test_image, model, inverse_label_map)
    print(f"Predicted plate: {result}")
    
    # Évaluation
    X_test = images.reshape(-1, 28, 28, 1) / 255.0
    y_test = np.array([inverse_label_map[label] for label in labels])
    evaluate_model(model, X_test, y_test)